In [6]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# must have maketable4.dta loaded into your local python environment or else python will not be able to path to the file 

In [9]:
import pandas as pd

# Read the old .dta file
old_file_path = 'maketable4.dta'
df = pd.read_stata(old_file_path)

# Save as the new version
new_file_path1 = "maketable4.dta"
df.to_stata(new_file_path1, write_index=False)

print("File updated successfully!")

File updated successfully!


In [10]:
data_table_4 = pd.read_stata(new_file_path1)
print(data_table_4)

    shortnam  africa  lat_abst  rich4    avexpr  logpgp95    logem4  asia  \
0        AFG     0.0  0.366667    0.0       NaN       NaN  4.540098   1.0   
1        AGO     1.0  0.136667    0.0  5.363636  7.770645  5.634789   0.0   
2        ARE     0.0  0.266667    0.0  7.181818  9.804219       NaN   1.0   
3        ARG     0.0  0.377778    0.0  6.386364  9.133459  4.232656   0.0   
4        ARM     0.0  0.444444    0.0       NaN  7.682482       NaN   1.0   
..       ...     ...       ...    ...       ...       ...       ...   ...   
158      YUG     0.0  0.488889    0.0  6.318182       NaN       NaN   0.0   
159      ZAF     1.0  0.322222    0.0  6.863636  8.885994  2.740840   0.0   
160      ZAR     1.0  0.000000    0.0  3.500000  6.866933  5.480639   0.0   
161      ZMB     1.0  0.166667    0.0  6.636364  6.813445       NaN   0.0   
162      ZWE     1.0  0.222222    0.0  6.000000  7.696213       NaN   0.0   

     loghjypl  baseco  
0         NaN     NaN  
1   -3.411248     1.0  
2  

In [11]:
pip install linearmodels

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.4 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 60.4/60.4 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.7 MB 4.1 MB/s eta 0:00:01
   ------------------- -------------------- 0.8/1.7 MB 8.6 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 15.9 MB/s eta 0:00:01
   ---------------- ----------------------- 1.1/2.8 MB 18.0 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.8 MB 14.0 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.8 MB 14.1 MB/s eta 0:00:01
   --------------

In [12]:
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS
import pandas as pd

# Remove rows with null values in 'avexpr', 'logpgp95', 'logem4', and 'loghjypl' variables
data_table_4 = data_table_4.dropna(subset=['avexpr', 'logpgp95', 'logem4', 'loghjypl'])

# Panels A and B, IV Regressions
# Columns 1 - 2 (Base Sample)
model1 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4]', data=data_table_4).fit()
model2 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4]', data=data_table_4).fit()

# Columns 3 - 4 (Base Sample w/o Neo-Europes)
model3 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4]', data=data_table_4[data_table_4['rich4'] != 1]).fit()
model4 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4]', data=data_table_4[data_table_4['rich4'] != 1]).fit()

# Columns 5 - 6 (Base Sample w/o Africa)
model5 = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4]', data=data_table_4[data_table_4['africa'] != 1]).fit()
model6 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + [avexpr ~ logem4]', data=data_table_4[data_table_4['africa'] != 1]).fit()

# Columns 7 - 8 (Base Sample with continent dummies)
data_table_4['other_cont'] = data_table_4['shortnam'].isin(['AUS', 'MLT', 'NZL']).astype(int)
model7 = IV2SLS.from_formula('logpgp95 ~ 1 + africa + asia + other_cont + [avexpr ~ logem4]', data=data_table_4).fit()
model8 = IV2SLS.from_formula('logpgp95 ~ 1 + lat_abst + africa + asia + other_cont + [avexpr ~ logem4]', data=data_table_4).fit()

# Column 9 (Base Sample, log GDP per worker)
model9 = IV2SLS.from_formula('loghjypl ~ 1 + [avexpr ~ logem4]', data=data_table_4).fit()

print(model1)
print(model2)
print(model3)
print(model4)
print(model5)
print(model6)
print(model7)
print(model8)
print(model9)

                          IV-2SLS Estimation Summary                          
Dep. Variable:               logpgp95   R-squared:                      0.2460
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2344
No. Observations:                  67   F-statistic:                    41.083
Date:                Sun, Feb 25 2024   P-value (F-stat)                0.0000
Time:                        20:27:05   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      2.2855     0.9399     2.4316     0.0150      0.4433      4.1277
avexpr         0.8845     0.1380     6.4096     0.00